# Modelado de los residuos restantes de los cristales
### Usamos Modeller


In [1]:
# Librerías
from prody import *
import csv
from prody import * 
from Bio import pairwise2, SeqIO
import os
import glob

/home/joel/anaconda3/envs/mds/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### Obtener la secuencia de UniProt
Utilizamos la secuencia UNIPROT correspondiente a la PROTEÍNA de interés.

In [2]:
# Nombre de la proteina de interes
prot_name = 'erk2'
# Secuencia P28482 (ERK2_HUMAN)
uniprot_id = "P28482"

In [3]:
# Secuencia de la CDK2 de UniProt
fasta = SeqIO.read(open(F'./data/{uniprot_id}.fasta'),'fasta')
seq_prot = str(fasta.seq)
print(seq_prot)

MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNVNKVRVAIKKISPFEHQTYCQRTLREIKILLRFRHENIIGINDIIRAPTIEQMKDVYIVQDLMETDLYKLLKTQHLSNDHICYFLYQILRGLKYIHSANVLHRDLKPSNLLLNTTCDLKICDFGLARVADPDHDHTGFLTEYVATRWYRAPEIMLNSKGYTKSIDIWSVGCILAEMLSNRPIFPGKHYLDQLNHILGILGSPSQEDLNCIINLKARNYLLSLPHKNKVPWNRLFPNADSKALDLLDKMLTFNPHKRIEVEQALAHPYLEQYYDPSDEPIAEAPFKFDMELDDLPKEKLKELIFEETARFQPGYRS


### Función find gaps
Cargamos la función que permite permite identificar los gaps en una secuencia y devolver el rango del gap.

In [4]:
import sys
sys.path.append(r'../..')
from modulos_python.find_gaps import find_gaps
# Recibe una secuencia (con gaps como "-") y un valor r para valorar el gap en +/- r posiciones 

In [5]:
# EJEMPLO:
find_gaps("---3-", r=0)

{'num_gaps': 2,
 'gap_lengths': [3, 1],
 'gap_list': [[1, 3], [5, 5]],
 'gap_window': [[1, 3], [5, 5]]}

### Obtenemos los PDB IDs de los cristales
Los ids los tomamos de la tabla de metadatos generada cen la libreta *Get_Metadatos..*.  

In [6]:
# Los IDs de los cristales están en el archivo
import pandas as pd
import glob
file_path = glob.glob(F"./data/TABLA_MTDATA_{prot_name.upper()}_*.csv")[0]
df_prot = pd.read_csv(file_path, index_col=1)
pdbids_list = df_prot.index
print("Total de PDB IDs a modelar:", len(pdbids_list))

Total de PDB IDs a modelar: 166


## Se ejecuta Modeller para refinamiento por loops
#### Descripción de los pasos a ejecutar:
- **1)** Se definen los directorios de entradas y salidas.
- **2)** Se carga la estructura cristalográfica a modelar (*cristal*), correspondiente a cada pdb id.
    - Si la proteína ya fue modelada se omite volver a modelarla.
- **3)** Se hace un **alineamiento de la secuencia** del cristal y la secuencia de Uniprot de la proteína, para ERK2 es **P28482**.
- **4)** Se **identifican los gaps en la secuencia del cristal**, si los hay. Se define una ventana de $\pm$ *r* residuos adyacentes al gap que se considerarán dentro del mismo. Con lo cual, si el gap tiene *n* residuos, su longitud final será de *n + 2r* residuos.
- **5)** Se modifica el método *select_atoms* de la clase *MyModel* de MODELLER para indicar que **únicamente modele los residuos que pertenecen al gap** del cristal, **dejando intacto el resto de los átomos de la proteína**. [Más información](https://salilab.org/modeller/manual/node23.html) sobre la metodología usada.
- **6)** Ejecuta MODELLER con los parámetros indicados, **generando únicamente un modelo**, el cual es guardado como _PDBID_**MODLL.pdb**. [Más información](https://salilab.org/modeller/9.21/manual/node19.html) sobre los parámetros utilizados.

In [7]:
# Directiros y parámetros base
struct_dir = F'../ARCHIVOS/CRISTALES/PROT_{prot_name.upper()}_CHAINS/'
tail_pdb = '_A.pdb' # extensión del archivo pdb del cristal
tail_model = '_MODLL' # epiteto del archivo del modelo final, la extensión .pdb la agrega modeller
pdbs_model_dir = F"../ARCHIVOS/CRISTALES/PROT_{prot_name.upper()}_MODELOS_modeller_NoPrep/" # Carpeta de salida para los modelos

# Creamos el directorio de salida si es que no existe
import os
if not os.path.exists(pdbs_model_dir):
    os.makedirs(pdbs_model_dir)

#### Creamos la función para ejecutar Modeller

In [23]:
from modeller import * # licencia modeller necesaria
from modeller.automodel import *

def run_modeller(pdb_code, num_res_window = 2, force_modeling = False,
                max_var_iterations = 500, repeat_optimization = 2):
    '''función para modelar gaps en estructuras cristalográficas utilizando Modeller'''
    ###########################
    if not force_modeling:
        if os.path.isfile(pdbs_model_dir + pdb_code + tail_model + ".pdb"):
            return print("El modelo ya existe en el directorio")
    ###########################
    try:
        # Se lee el archivo pdb. La selección omite residuos negativos.
        # Además omite los residuos no estandar, o con modificaciones postraduccionales como el TPO (Tirosina fosfatada)
        stc_prot = parsePDB(struct_dir 
                            + pdb_code + tail_pdb).select('not nonstdaa and resid > 0') 
        chaid = stc_prot.getChids()[0] # Obtiene el id de la cadena
        ref_hv = stc_prot.getHierView()[chaid] # Se obtiene sólo la cadena
        seq_cry = ref_hv.getSequence() # Se obtiene la secuencia de la estructura cristalográfica   
    except:
        return print("Error al abrir y modelar:", pdb_code)
    ########################### 
    # Pregunta si la longitud y la secuencias de Uniprot y de la estructura son iguales
    same_seq = len(seq_cry) == len(seq_prot) and seq_cry == seq_prot
    # si same_seq es verdadero, se omite hacer el modelado pues la estructura está completa
    if same_seq:
        return print("La proteína " + pdb_code + " ya está completa") 
        # Si le hacen falta átomos a un residuo fuera del gap, esto se corregirá 
        # con PDB2PQR y pdb4amber en un fase posterior al modelado.
    ###########################     
    print("Modelando proteína " + pdb_code)
    # Obtener la secueNcia de la estructura a modelar y guardarla en un archivo.
    # Alineamiento global, se penalizan con -10 la abertura de nuevos gaps, se obtiene el mejor alineamiento.
    alignment = pairwise2.align.globalxs(seq_cry, seq_prot, -10, -0.5, gap_char='-')[0]
    # Secuencias del alineamiento
    algn1_struc = alignment[0] # Secuencia alineada de la estructura cristalográfica
    algn2_seq = seq_prot # Secuencia completa de UniProt

    # Nombre de los cabezales de las secuencia deben coincidir con el de los archivos de entrada y salida
    crys_file_name = pdb_code + tail_pdb
    model_file_name = pdb_code + tail_model

    ''' NECESARIO: There should be 10 fields separated by colons ":".
    Please check the file to make sure your sequences end with the '*' character.
    Nomenclaturas de los campos del header: 
    https://salilab.org/modeller/8v2/manual/node176.html'''

    # HEADERS (la secuencia del cristal va primero, luego la secuencia completa)
    struc_header = "structureX:" + crys_file_name + ":.:" + chaid + ":.:" + chaid + ":.:.:.:"
    seq_header = "sequence:" + model_file_name + ":.:.:.:.:.:.:.:"
    
    # Crea el archivo de alineamiento con la estructura requerida por modeller
    alg_filename = pdb_code + ".alg"
    with open(alg_filename, "w") as handle:
        handle.write("\n>P1;%s\n%s\n%s*\n>P1;%s\n%s\n%s*\n" % (crys_file_name, struc_header, algn1_struc, 
                                                               model_file_name, seq_header, algn2_seq))
    ##########################
    # Ejecuta find_gaps para obtener los posibles gaps de la secuencia
    gaps = find_gaps(algn1_struc, r = num_res_window)
    num_gaps = gaps["num_gaps"]
    gap_i = gaps["gap_window"]
    
    # Se crea el string que necesita Modeller para definir al primer gap
    s = "self.residue_range('" + str(gap_i[0][0]) + "', '" + str(gap_i[0][1]) + "')"

    # A pesar de que sólo haya un gap, se ejecuta el ciclo, 
    # si hay más de uno, se extiende el String para incluir los demás
    for i in range(1, num_gaps):
        s = s + ", " + "self.residue_range('" + str(gap_i[i][0]) + "', '" + str(gap_i[i][1]) + "')" #nótese la coma
    ##########################
    ''' SE EJECUTA MODELLER'''
    ##########################
    # Se crea un nuvo ambiente de modeller
    env = environ()
    env.io.atom_files_directory = ['.', struct_dir]
    
    # Se modifica la clase MyModel de Modeller para fijar los residuos ya existentes en el cristal
    # Estos residuos no se modelarán ni sus átomos cambiarán de posición
    # NECESARIO para poder capturar los valores obtenidos por el rango de gaps
    
    MyModel_code = """
class MyModel(automodel):
    def select_atoms(self):
        return selection(""" + s + """)
""" # Al estar dentro del loop la identación de este string es importante
    exec(MyModel_code, globals()) # Se lleva a cabo la modificación a MyModel para agregar la región fijada
    # Se intancia el objeto MyModel
    a = MyModel(env, alnfile = alg_filename, # Lee el archivo fasta creado y guardado en el directorio actual
                      knowns = crys_file_name, # Archivo pdb crys, que coincide con el id en el archivo fasta
                      sequence = model_file_name) # Nombre del modelo
    a.starting_model= 1
    a.ending_model  = 1
    # Se definen los parámetros de refinamiento dle modelo:
    # Más info sobre el refinamiento: https://salilab.org/modeller/9.21/manual/node19.html
    a.library_schedule = autosched.slow # Originalmente comentado
    a.max_var_iterations = max_var_iterations #*500
    a.md_level = refine.slow # Nivel del refinamiento
    a.repeat_optimization = repeat_optimization # Número de repeticiones para la optimización
    a.make()

    ###########################
    # Renombra el archivo pdb a pdb_code + _full.pdb
    # DEBE ser el único archivo pdb en el directorio '.'
    model_file = glob.glob('./' + pdb_code + '*.pdb')[0] # Nombre del archivo pdb
    os.rename(model_file, pdbs_model_dir + pdb_code + tail_model + ".pdb") # Mueve el pdb a la carpeta model_pdbs
    # Elimina los archivos extra generados - Todos empiezan con el codigo del PDB ID
    for f in glob.glob(pdb_code + "*"):
        os.remove(f)

### Ejecutamos Modeller

In [32]:
import time
start = time.time()

# se lleva a cabo el modelado de todas las estructuras cristalográficas
# for pdb_code in pdbids_list:
run_modeller("3i60", 
             num_res_window = 2, 
             force_modeling = True,
             max_var_iterations = 1000, 
             repeat_optimization = 2)

end = time. time()
print(end - start)

@> 2734 atoms and 1 coordinate set(s) were parsed in 0.03s.


Modelando proteína 3i60

check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 angstroms:

ALN_POS  TMPL  RID1  RID2  NAM1  NAM2     DIST
----------------------------------------------
    188     1  182   187      V     A   12.239
END OF TABLE
read_to_681_> topology.submodel read from topology file:        3
mdtrsr__446W> A potential that relies on one protein is used, yet you have at
              least one known structure available. MDT, not library, potential is used.
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies
condens_443_> Restraints marked for deletion were removed.
              Total number of restraints before, now:    37141    34495
256 (of 2915 total) atoms selected for optimization
iupac_m_397W> Atoms were not swapped because of the uncertainty of

46.41107964515686


### Estructuras que en la primera ronda tuvieron problemas con el loop A


In [29]:
num_struct = [16, 34, 35, 36, 38, 39, 54, 61, 64, 86, 90, 91, 92, 154]
pdbids_list[num_struct]

Index(['3i60', '4fv4', '4fv5', '4fv6', '4fv8', '4fv9', '4o6e', '4qp8', '4qta',
       '5ax3', '5bvd', '5bve', '5bvf', '6ots'],
      dtype='object', name='PDB_ID')

Todas estas estructuras fueron insatisfactorias porque el loop A de la proteína, ausente en todas ellas, se entrelazaba con otro loop menor, también ausente.

In [44]:
import pytraj as pt
import nglview as nv

ejemplo = pt.load(F'{pdbs_model_dir}/modelos_descartados/3i60_MODLL.pdb')
view = nv.show_pytraj(ejemplo)
view.clear_representations()
view.add_cartoon(color='white')
view.add_cartoon(selection = '175-185', color='red')
view.add_cartoon(selection = '199-207', color='blue')
view

NGLWidget()

Para correfir esto, en una segunda ronda fueron "corregidas" estas estructuras tras establecer la ventana del gap a r = 1 residuo, aumentar el refinamiento con Modeller, y agregar un C$\alpha$ "dummy" del residuo 182 que forma parte del loop A de la estructura 1wzy, para "forzar" a Modeller a que orientase el loop A  en dirección opuesta al loop menor. 